#Applied Natural Language Processing 955G5

##Computer Based Examination, 2026

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [33]:
# update your candidate number here
candidate_number = 291065

#Question 2 (50 Marks)

This question is about NER-tagging and Naive Bayes classification.

In [47]:
### do not change the code in this cell
# make sure you run this cell
import nltk
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.probability import FreqDist



# This is a list of sentences on various topics.
sentences=["Treasury Secretary Scott Bessent made his case for President Donald Trump’s global tariffs after Trump and Xi Jinping agreed a trade deal.",
           "Following a $100 billion investment from Nvidia, OpenAI agreed to pay Oracle $300 billion over five years for computer infrastructure.",
           "At a meeting of leaders in Beijing, Russia thanked North Korea for the courage of their troops fighting in Ukraine."]

# This is a dictionary containing counts of NER tags from a corpus of sentences which were labelled as topics A and B.
classtagcounts={"A":{"GPE":36, "PERSON":64},
                "B":{"ORG":73, "MONEY":27}}

# This is a list of NER tags.
taglist = ["GPE","PERSON","ORG","MONEY","DATE",""]


a) By following the steps below, tag the sentences with named entity types and construct a bag-of-tags representation of each one.

i) Apply the function `nlp()`, which has been imported above, to each sentence in turn. From the output, produce a list, named `tagged`, of lists of tuples containing token, NER tag pairs extracted from the output of `nlp()`.

So, for example, the list `["Skyscanner started in Edinburgh"]` would become `[[('Skyscanner', 'ORG'), ('started', ''), ('in', ''), ('Edinburgh', 'GPE')]]`.

(7 marks)

In [48]:
tagged = [] 

for sent in sentences:
    _sent = nlp(sent)
    _sent_tagged = [(token.text, token.ent_type_) for token in _sent]
    tagged.append(_sent_tagged)

print(tagged[1])

[('Following', ''), ('a', ''), ('$', 'MONEY'), ('100', 'MONEY'), ('billion', 'MONEY'), ('investment', ''), ('from', ''), ('Nvidia', 'GPE'), (',', ''), ('OpenAI', 'ORG'), ('agreed', ''), ('to', ''), ('pay', ''), ('Oracle', 'ORG'), ('$', 'MONEY'), ('300', 'MONEY'), ('billion', 'MONEY'), ('over', ''), ('five', 'DATE'), ('years', 'DATE'), ('for', ''), ('computer', ''), ('infrastructure', ''), ('.', '')]


ii) Convert each list of word, tag pairs into a bag-of-tags representation using the FreqDist class, which has been imported above.

So, for example `[[('Skyscanner', 'ORG'), ('started', ''), ('in', ''), ('Edinburgh', 'GPE')]]` would become `[FreqDist({'ORG': 1, '': 2, 'GPE': 1})]`.

(5 marks)

In [49]:
BoT=[]

for sent in tagged:
    tags = [tag for token,tag in sent]
    BoT.append(FreqDist(tags))

BoT

[FreqDist({'': 16, 'PERSON': 7, 'ORG': 1}),
 FreqDist({'': 13, 'MONEY': 6, 'ORG': 2, 'DATE': 2, 'GPE': 1}),
 FreqDist({'': 17, 'GPE': 5})]

b) In this part of the question we will consider the ideas behind Naive Bayes, calculate the parameters of a model and predict the probability of each sentence being a member of classes A and B.

i) The form of the Naive Bayes model is based on the idea that we want to find the class that maximises $p(class|document)$.

Explain how Bayes theorem is applied to this and what naive assumption is made about the features in the document. Describe the priors and conditional probabilities that are used to predict the most likely class for a document.

(5 marks)

---

Bayes Theorem is a framework which enables us to derive the most probable class outcomes given a sequence of input features. In our class we want to find the class is most probable for a given input document. 

In order to understand the idea behind the Bayes we first need to instantiate our problem. We have many data instances which are documents. A document is a data structure which is made up of a sequence of features with a given order (words, punctuation, numbers). We can represent this as a tuple of features. We use a tuple because the features are indexed in a given order so it retains the format of a sentence or paragraph:

$$ f_1^2 = (f_1, f_2,...,f_n) $$

We want to assign our documents a class, this is something that they belong, i.e. categorizing articles in a newspaper as "football", "politics" etc. With this, there exists a possible set of classes $C$. An individual class from within the set is denoted $c$ which can be denoted $c \in C$. 

The goal of a "model" is to look at the features $f_1^2$ and assign a class based on the features. If the number of classes $C > 1$ then the selection of a class with be derived based on the class which is most likely, this is written $P(c|f_1^2)$

Bayes Theorem provides us with a framework to execute this allocation. 

$$\operatorname*{argmax}_{c} P(c|f_1^n) = \operatorname*{argmax}_{c} \frac{P(f_1^n|c) \cdot P(c)}{P(f_1^n)}$$

This equation is made up of 5 parts: 
- Prior
- Conditional Probability/Likelihood
- Evidence 
- Posterior Probability
- Argmax

The Prior $P(c)$ is the distribution of the class across all instances. If there are all 100 pieces of data, 50 are labelled X and 50 labelled Y then `Class X = 0.5` and `Class Y = 0.5`. In the structure of Bayes, it tells of the before information. Before we know anything about a piece of data, what are the global assignments of classes. 

The Conditional Probability/Likelihood $P(f_1^n|c)$ says, given we know the class of something, what is the max probability that it comes from some set of features. In Bayes it represents the update mechanism. Now that we have an actual data instance, we use it to improve and inform the prior distribution. 

The Evidence $P(f_1^2)$ is probability that a data instance is in the set of all possible feature sets. It should be noted that this probability is the same for all feature sets, therefore, for Bayes, this is just a normalization term. It ensures that all final probabilities sum to one. However, in terms of ratios, it doesn't do anything. The highest result will still be the most likely even if the number computed isn't a real probability. As a result this term is often dropped for simplicity. 

On the left hand side of the equation is the posterior $P(c|f_1^n)$. This is the outcome of updating the prior with the conditional probability. It tells us, given we have seen a particular feature set, what class is it most likely from. 

Finally, there is argmax. This tells from all of the possible feature sets, select the one that is most likely. 

In practice, it is generally impossible to calculate Bayes in its true form. This is because a document is a sequence of words joined together is a very specific order. Word themselves are sparse features, meaning compared to the possibility of all words in our vocab, the possibility of seeing a particular word is low. A sentence is therefore an extremely sparse vector as its need to compute the probability of not only seeing a rare word but probability of seeing it the exact sequence the sentence places it in. Calculating the joint probability $P(f_1, f_2, ..., f_n|c)$ of a sparse vector like this leads an absurdly small probability, this is known as the curse of dimensionality. Given that the feature probabilities are themselves low and the length of documents/sentence, the calculated probability of an exact sentence will be 0. 

The naive assumption allows us to treat the input features, the sentence, as independent features. This means that the format of the sequence is dropped, i.e. the current word does not depend on the previous. Whilst this is incorrect, it simplifies the calculation and allows us to derive a probability. In practice, it also means that we exchange the sentence data structure from something with order, like a list or tuple, to a something unorder, like a dictionary, i.e. a Bag-of-Words

$$P(f_1, ..., f_n|c) \approx P(f_1|c) \times P(f_2|c) \times ... \times P(f_n|c)$$

---


ii) Sum the counts in `classtagcounts` to get total frequencies for classes `A` and `B` and put the results in a dictionary called `classcounts`.

Then create a dictionary, called `classpriors`, containing the prior probabilities for each class based on the frequencies in the dictionary `classcounts`.

(7 marks)

In [50]:
classcounts = {}

for c in classtagcounts.keys():
    classcounts[c] = sum(classtagcounts[c].values())

classpriors = {}
t = sum(classcounts.values())

for _c in classcounts.keys():
    classpriors[_c] = classcounts[_c]/t
 
classpriors

{'A': 0.5, 'B': 0.5}

iii) Define a function, `condprobs()`, which will take two inputs - a dictionary mapping class labels to dictionaries of feature frequencies and a dictionary mapping class labels to frequencies - and produce a dictionary of dictionaries containing conditional probabilities. Each key in the outer dictionary should be a class label, the inner keys should be features and the inner values should be conditional probabilities: $$p(feature|label)$$.

Apply this function to the `classtagcounts` and `classcounts` dictionaries defined above.

(5 marks)

In [58]:
def condprobs(dict_tf, dict_c):
    cp = {}
    for c in dict_c.keys():
        _t = dict_c[c]
        
        current_values = dict_tf.get(c,{})
        current_values_cp = {}

        for label in current_values:
            current_values_cp[label] = current_values[label]/_t

        cp[c] = current_values_cp

    return cp

conditional_probabilities = condprobs(classtagcounts, classcounts)

print(conditional_probabilities)

{'A': {'GPE': 0.36, 'PERSON': 0.64}, 'B': {'ORG': 0.73, 'MONEY': 0.27}}


iv) Explain why we might want to smooth the conditional probabilities, and how add one smoothing works.

(5 marks)

Smoothing is a common tool that we use to address the sparsity experienced in NLP applications. In order to calculate the probability using Naive Bayes we need to update the prior probabilities with the features conditional probabilities. A problem arises when features arise in the test or deployed data that were not seen in the training data. If something was not seen in the training data then it will have a conditional probability of 0. Due to the independence assumption, and Bayes Theorem, we take the joint probability of all features in the document $P(f_1|c) \times P(f_2|c) \dots$, if 1 document has a zero probability then the total outcome for the document will also be 0. By definition, this cannot be true as we are seeing both the feature and the document in our processing. 

Add-one smoothing works by initialising all known features, whether it be words or tags, with a count of 1. This avoids there being any 0 probability features calculated. In our case, we know from the tag list that there are 6 possible tags `"GPE","PERSON","ORG","MONEY","DATE", ""` = ($|V|$), however in the training data and `classtagcounts`, we only see `GPE,PERSON,ORG,MONEY` and more importantly, we only see 2 tags in either class `A` or class `B`. This means that is a `MONEY` tag, which we see in class `B` were to come in up class `B`, it would result in a probability of 0, even though we know the tag can exist in the corpus. Additionally, because we are adding one to all known instances of classes ($|V|$) we need to adjust our overall Naive Bayes conditional probabilities denominator to account for the `n` additional counts. 


$$P(feature|class) = \frac{count(feature, class) + 1}{TotalCount(class) + |V|}$$


v) Using the list of possible tags, `taglist`, apply add one smoothing to the `classtagcounts` dictionary and update the `classcounts` dictionary to reflect the modified class frequencies.

(5 marks)

In [56]:
smoothed_classtagcounts = {}

for key in classtagcounts.keys():
    current = classtagcounts.get(key,{})
    smooth_current = {}
    print(current)
    for tag in taglist:
        orig_count = current.get(tag,0)
        smooth_current[tag] =  orig_count + 1
    smoothed_classtagcounts[key] = smooth_current

smoothed_classtagcounts

{'GPE': 36, 'PERSON': 64}
{'ORG': 73, 'MONEY': 27}


{'A': {'GPE': 37, 'PERSON': 65, 'ORG': 1, 'MONEY': 1, 'DATE': 1, '': 1},
 'B': {'GPE': 1, 'PERSON': 1, 'ORG': 74, 'MONEY': 28, 'DATE': 1, '': 1}}

vi) Call the `condprobs` function on the smoothed frequencies.

(1 mark)

In [59]:
smoothed_conditional_probabilities = condprobs(smoothed_classtagcounts, classcounts)
print(conditional_probabilities)
print(smoothed_conditional_probabilities)

{'A': {'GPE': 0.36, 'PERSON': 0.64}, 'B': {'ORG': 0.73, 'MONEY': 0.27}}
{'A': {'GPE': 0.37, 'PERSON': 0.65, 'ORG': 0.01, 'MONEY': 0.01, 'DATE': 0.01, '': 0.01}, 'B': {'GPE': 0.01, 'PERSON': 0.01, 'ORG': 0.74, 'MONEY': 0.28, 'DATE': 0.01, '': 0.01}}


vii) Use the class priors and conditional probabilities you have just calculated to derive probabilities of belonging to the classes A and B for each bag-of-tags representation you created in a)ii). For each item, predict the most likely class and print out each original sentence alongside your prediction.

(10 marks)

In [86]:
for i,bag in enumerate(BoT):
    scores = {}

    for label in ["A","B"]:
        score = classpriors[label]
        for tag,freq in bag.items():
            if tag in smoothed_conditional_probabilities[label]:
                prob = smoothed_conditional_probabilities[label][tag]
                score *= (prob ** freq)
        scores[label] = score

    prediction = "A" if scores["A"] > scores["B"] else "B"

    print(f"sentence {i}")
    print(f"label A: {scores["A"]}, label B: {scores["B"]}")
    print(f"there most likely class is {prediction}")
    print(f'"{sentences[i]}"')
    print("---")

sentence 0
label A: 2.4511139453125018e-36, label B: 3.700000000000002e-47
there most likely class is A
"Treasury Secretary Scott Bessent made his case for President Donald Trump’s global tariffs after Trump and Xi Jinping agreed a trade deal."
---
sentence 1
label A: 1.850000000000001e-47, label B: 1.3194156523520012e-36
there most likely class is B
"Following a $100 billion investment from Nvidia, OpenAI agreed to pay Oracle $300 billion over five years for computer infrastructure."
---
sentence 2
label A: 3.4671978500000007e-37, label B: 5.000000000000003e-45
there most likely class is A
"At a meeting of leaders in Beijing, Russia thanked North Korea for the courage of their troops fighting in Ukraine."
---
